In [ ]:
# -*- coding: utf-8 -*-
# مشروع تحليل المشاعر باللهجة السعودية البيضاء
# إعداد: أمل البريكي
# الغرض: إنشاء نموذج ذكاء اصطناعي لتحليل المشاعر في النصوص السعودية

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

# ------------------------------
# 1. إنشاء بيانات بسيطة باللهجة السعودية البيضاء
# ------------------------------

data = {
    "النص": [
        "الخدمة ممتازة والله",
        "التطبيق بطئ مرة",
        "السعر مناسب جدًا",
        "ما انصح احد بالتعامل معهم",
        "عادي مو ذاك الزود",
        "التوصيل سريع ومريح",
        "واجهت مشاكل في الدفع",
        "الدعم الفني متعاون",
        "تجربة سيئة بصراحة",
        "مره اعجبني التعامل"
    ],
    "المشاعر": [
        "إيجابي",
        "سلبي",
        "إيجابي",
        "سلبي",
        "محايد",
        "إيجابي",
        "سلبي",
        "إيجابي",
        "سلبي",
        "إيجابي"
    ]
}

df = pd.DataFrame(data)
print("نظرة على البيانات:")
print(df.head())

# ------------------------------
# 2. تجهيز البيانات للنموذج
# ------------------------------

X = df["النص"].values
y = df["المشاعر"].values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# ------------------------------
# 3. تحويل النصوص إلى أرقام
# ------------------------------

tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
padded = pad_sequences(sequences, maxlen=10, padding="post")

# ------------------------------
# 4. تقسيم البيانات
# ------------------------------

X_train, X_test, y_train, y_test = train_test_split(padded, y_categorical, test_size=0.2, random_state=42)

# ------------------------------
# 5. بناء النموذج
# ------------------------------

model = Sequential([
    Embedding(input_dim=1000, output_dim=16, input_length=10),
    LSTM(64, return_sequences=False),
    Dense(32, activation="relu"),
    Dense(3, activation="softmax")  # ثلاث فئات: إيجابي، سلبي، محايد
])

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# ------------------------------
# 6. تدريب النموذج
# ------------------------------

history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), verbose=1)

# ------------------------------
# 7. تجربة النموذج
# ------------------------------
def توقع_المشاعر(نص):
    تسلسل = tokenizer.texts_to_sequences([نص])
    مبطن = pad_sequences(تسلسل, maxlen=10, padding="post")
    النتيجة = model.predict(مبطن)
    الفئة = np.argmax(النتيجة)
    التسمية = label_encoder.inverse_transform([الفئة])[0]
    print(f"النص: {نص}")
    print(f"التصنيف: {التسمية}")

# اختبار النموذج
توقع_المشاعر("الخدمة كانت سريعة وممتازة")
توقع_المشاعر("مره تأخر التوصيل وما رديتو")
توقع_المشاعر("عادي الخدمة متوسطة")

